# CHIPS-CVN Example Notebook

## Get Example ROOT Hit Maps

Firstly we must download some data to work with. We will download some beam $\nu_{e}$ and $\nu_{\mu}$ charged-current (CC), quasi-elastic (QEL) events. There are 20000 events for each, which will be stored in ./example_data (everything in this notebook assumes this relative path).

In [1]:
!mkdir example_data
!wget -q --show-progress -P example_data http://www.hep.ucl.ac.uk/~jtingey/beam_nuel_qel_cc.tar
!tar -C example_data -xf example_data/beam_nuel_qel_cc.tar
!rm example_data/beam_nuel_qel_cc.tar
!wget -q --show-progress -P example_data http://www.hep.ucl.ac.uk/~jtingey/beam_numu_qel_cc.tar
!tar -C example_data -xf example_data/beam_numu_qel_cc.tar
!rm example_data/beam_numu_qel_cc.tar

beam_nuel_qel_cc.ta 100%[===================>] 230.23M   112MB/s    in 2.1s    
beam_numu_qel_cc.ta 100%[===================>] 217.92M   112MB/s    in 1.9s    


## Setup the GPUs

We need to first initialise the GPUs

In [2]:
import chipscvn.utils as utils
utils.gpu_setup()  # Setup the GPU's

---  4 Physical GPUs, 4 Logical GPUs ---


## Convert ROOT Hit Maps to tfrecords

We first convert the ROOT hit map files into .tfrecords files, so Tensorflow (TF) can load them on-the-fly when training. These contain serialised "examples" containing "features" and "labels". We also split the data into TRAIN, VALIDATION and TEST sets. The DataCreator class in the data module does this (by default it does this in parralel)

In [3]:
import chipscvn.data as data

# Firstly beam_nuel_qel_cc (input_directory, geometry_name, test/val fraction, join, single_thread)
print("Processing nuel events...")
nuel_creator = data.DataCreator("./example_data/beam_nuel_qel_cc", "chips_1200_sk1pe", 0.1, 10, True)
nuel_creator.preprocess()

# Secondly beam_numu_qel_cc (input_directory, geometry_name, test/val fraction, join, single_thread)
print("Processing numu events...")
numu_creator = data.DataCreator("./example_data/beam_numu_qel_cc", "chips_1200_sk1pe", 0.1, 10, True)
numu_creator.preprocess()

Processing nuel events...
Processing job 0...
Processing numu events...
Processing job 0...


## The Configuration File

Everything in chips-cvn is controlled by a YAML (YAML Ain't Markup Language) configuration file, which is similar to JSON. This makes dealing with the large amount of possible parameters much, much easier. The file is split into different sections for the different functional sections of the code.

In [4]:
!cat config/example_energy_estimation.yml  # Look at the example energy estimation configuration file

---
exp: # Variables that define the experiment
    name: "energy_estimation_example"
    experiment_dir: "experiments/"
data: # Variables used to load the data
    input_dirs: 
        - "example_data/beam_nuel_qel_cc/tf/chips_1200_sk1pe/"
    img_shape: [64, 64, 3]
    batch_size: 64
    max_examples: 2500
model: # Variables used in building the model
    name: "single_par"
    parameter: "lepEnergy"
    lr: 0.001
    dense_units: 256
    dropout: 0.0
    kernel_size: 3
    filters: 64
trainer: # Variables used by the trainer
    num_epochs: 5
    es_delta: 5
    es_epochs: 5
study: # We list the number of trials and then all the variables we want to tune
    trials: 200
    data:
        batch_size: [32, 64, 128]
    model:
        lr: [0.0005, 0.01]
        dense_units: [64, 128]
        dropout: [0.0, 0.5]
        kernel_size: [3, 5]
        filters: [32, 64, 128]
eval: # Variables used by the evaluator
    input_dirs: 
        - "example_data/beam_nuel_qel_cc/tf/chips_1200_sk1pe/

## Look at an Event

To inspect some example events we will use the "DataLoader" from the data module that creates TF datasets from the .tfrecords files and just look at a few events and what they contain. Each "example" in the dataset is a tuple of two things the "image" and the "labels".

In [5]:
import matplotlib.pyplot as plt
import chipscvn.config as config

conf = config.process_config("config/example_energy_estimation.yml")  # Load the example configuration
data_loader = data.DataLoader(conf)  # Setup the DataLoader using the configuration

dataset = data_loader.train_data()  # Get the training dataset from the DataLoader
for example in dataset.take(1):  # Dataset is iterable, take first batch of 64 examples
    print("Batch type: " + str(type(example)) + ", Elements: " + str(len(example)))
    print(str(example[0].shape))  # Print the image tensor shape
    print(example[1].keys())  # Print the labels dictionary keys

Batch type: <class 'tuple'>, Elements: 2
(64, 64, 64, 3)
dict_keys(['pdg', 'type', 'vtxX', 'vtxY', 'vtxZ', 'dirTheta', 'dirPhi', 'nuEnergy', 'lepEnergy'])


In [6]:
for example in dataset.take(1):  # Dataset is iterable, take first batch of 64 examples
    image = example[0][0].numpy()  # Get the first "image" in the batch
    charge, time, hough = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    
    plt.figure(1, figsize=(15, 7))
    plt.subplot(131)
    plt.title('charge channel')
    plt.imshow(charge)
    plt.colorbar(orientation='horizontal')

    plt.subplot(132)
    plt.title('time channel')
    plt.imshow(time)
    plt.colorbar(orientation='horizontal')
    
    plt.subplot(133)
    plt.title('hough channel')
    plt.imshow(hough)    
    plt.colorbar(orientation='horizontal')
    
    plt.show()

## Train a Simple Energy Estimation Network

As a first example we will train a simple Convolutional Neural Network to estimate the $\nu_{e}$ energy in $\nu_{e}$ QEL-CC events.

In [7]:
import chipscvn.config as config
import chipscvn.data as data
import chipscvn.models as models
import chipscvn.trainers as trainer

conf = config.process_config("config/example_energy_estimation.yml") # Load the nuel configuration
data_loader = data.DataLoader(conf)                                  # Setup the TF datasets
model = utils.get_model(conf)                                        # Create the model
model.summarise()                                                    # Print model summary

model_trainer = utils.get_trainer(conf, model, data_loader)          # Setup the model train
model_trainer.train()                                                # Train the model

Model: "single_par_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 128)  

## Test the train model

Whenever you run a training "experiment" chips-cvn creates a directory containing various outputs in the ./experiments directory.

- Model weights, saved at the end of each epoch
- Tensorboard files
- Image of the model structure

We can now test the model on the test dataset by loading the trained model and evaluating its performance. We will not load the model here as we already have it fully trained in memory...

In [9]:
import ROOT
from root_numpy import fill_hist
import chipscvn.evaluators as evaluators
%jsroot on

evaluator = utils.get_evaluator(conf)          # Setup the model train
evaluator.run()                                                # Train the model
labels, predictions = evaluators.predict(data_loader.test_data(), model)
diff = labels['lepEnergy'] - predictions['lepEnergy']
diff = diff.to_numpy()
h = ROOT.TH1F("resolution_h","resolution",50,-800,800)
fill_hist(h, diff)
c = ROOT.TCanvas("resolution_c","resolution",800,600)
h.Draw()
c.Draw()

Welcome to JupyROOT 6.18/04
     30/Unknown - 0s 13ms/step - loss: 126482.9531 - mae: 240.2522 - mse: 126482.9531[126482.953125, 240.25217, 126482.95]
--- 1.015367031097412 seconds to test model ---
